In [65]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import sys
import os
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull

current_path = os.path.dirname(os.path.abspath('__file__'))

project_root = os.path.dirname(current_path)

if project_root not in sys.path:
    sys.path.append(project_root)
from stage2_scripts.finetune import load_data as load_temp_data
from stage3_scripts.finetune_do import load_data as load_do_data  
from utils.utils import load_pd_data, combine_dataset

# seed = 40

ids = pd.read_csv('../utils/intersection_ids.csv')
ids = ids['nhdhr_id'].to_list()

label_names = 'obs_do'

In [66]:
def get_data(lake_id, model_type, datetime, stage, depth):
    seed = 40
    if model_type == 'transformer':
        seed = 44
    if stage == 3:
        read_path = os.path.join(f'../stage{stage}_results/{model_type}/{datetime}/' + f'{lake_id}/{seed}', 'pred_test_raw.npy')
    else:
        read_path = os.path.join(f'../stage{stage}_results/{model_type}/{datetime}/' + f'{lake_id}/{seed}', 'pred_test.npy')
    pred = np.load(read_path)

    read_path = os.path.join(f'../stage{stage}_results/{model_type}/{datetime}/' + f'{lake_id}/{seed}', 'obs_test.npy')
    obs = np.load(read_path)
    read_path = os.path.join(f'../stage{stage}_results/{model_type}/{datetime}/' + f'{lake_id}/{seed}', 'sim_test.npy')
    sim = np.load(read_path)
    read_path = os.path.join(f'../stage{stage}_results/{model_type}/{datetime}/' + f'{lake_id}/{seed}', 'test_dates.npy')
    dates = np.load(read_path)

    indices = list(range(0, pred.shape[0], depth))
    pred = pred[indices, :]
    obs = obs[indices, :]
    sim = sim[indices, :]
    dates = dates[indices, :]


    return pred, obs, sim, dates

In [67]:
from scipy.interpolate import splprep, splev
def smooth_boundary(points, hull, smoothing_factor=1):
    hull_points = points[hull.vertices]
    hull_points = np.append(hull_points, [hull_points[0]], axis=0)  # Close the loop
    tck, u = splprep([hull_points[:, 0], hull_points[:, 1]], s=smoothing_factor, per=True)
    unew = np.linspace(0, 1.0, 1000)
    out = splev(unew, tck)
    return out

In [ ]:
from shapely.geometry import Point, MultiPoint
from shapely.ops import cascaded_union, polygonize
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay

def alpha_shape(points, alpha):
    """
    Compute the alpha shape (concave hull) of a set of points.
    :param points: Iterable container of points.
    :param alpha: Alpha value to influence the gooeyness of the border. Smaller
                  numbers don't fall inward as much as larger numbers.
    :return: List of (x, y) coordinates of the alpha shape.
    """
    if len(points) < 4:
        # When you have a triangle, there is no sense in computing an alpha shape.
        return MultiPoint(list(points)).convex_hull
    
    def add_edge(edges, edge_points, coords, i, j):
        """ Add a line between the i-th and j-th points, if not in the list already """
        if (i, j) in edges or (j, i) in edges:
            # already added
            return
        edges.add( (i, j) )
        edge_points.append(coords[ [i, j] ])
    
    coords = np.array([point for point in points])
    tri = Delaunay(coords)
    edges = set()
    edge_points = []
    # Loop over triangles:
    # ia, ib, ic = indices of corner points of the triangle
    for ia, ib, ic in tri.vertices:
        pa = coords[ia]
        pb = coords[ib]
        pc = coords[ic]
        
        # Lengths of sides of triangle
        a = np.linalg.norm(pa-pb)
        b = np.linalg.norm(pb-pc)
        c = np.linalg.norm(pc-pa)
        
        # Semiperimeter of the triangle
        s = (a + b + c)/2.0
        
        # Area of the triangle by Heron's formula
        area = np.sqrt(s*(s-a)*(s-b)*(s-c))
        if area == 0:
            circum_r = 0
        else:
            circum_r = a*b*c/(4.0*area)
        
        # Here's the radius filter.
        if circum_r < 1.0/alpha:
            add_edge(edges, edge_points, coords, ia, ib)
            add_edge(edges, edge_points, coords, ib, ic)
            add_edge(edges, edge_points, coords, ic, ia)
    
    m = MultiPoint([Point(point) for point in coords])
    triangles = [Polygon(edge) for edge in polygonize(edge_points)]
    return cascaded_union(triangles), edge_points

In [ ]:
for index, lake_id in enumerate(ids):
    try:
        if lake_id != 'nhdhr_120019016':
            continue
        test_pd_data = load_pd_data(
            ids=[lake_id], dataset='test', save_path='../data/processedByLake/')['temp_data']
        unique_depths = np.unique(test_pd_data['vertical_depth'])
        n_depth = len(unique_depths)
        print("n_depth:", n_depth)
        # temperature

        lstm_pred_temp, obs_temp, sim_temp, dates_temp = get_data(lake_id, 'lstm', '2024-08-08-21-10', stage = 2, depth=n_depth)

        pgfm_pred_temp, _, _, _ = get_data(lake_id, 'fm-pg', '2024-08-09-03-10', stage = 2, depth=n_depth)

        ealstm_pred_temp, _, _, _ = get_data(lake_id, 'ealstm', '2024-08-10-22-30', stage = 2, depth=n_depth)
        transformer_pred_temp, _, _, _ = get_data(lake_id, 'transformer', '2024-08-10-01-35', stage = 2, depth=n_depth)
        
        # oxygen
        lstm_pred_do, obs_do, sim_do, dates_do = get_data(lake_id, 'lstm', '2024-08-09-23-00', stage = 3, depth=n_depth)

        pgfm_pred_do, _, _, _ = get_data(lake_id, 'fm-pg', '2024-08-10-04-14', stage = 3, depth=n_depth)

        ealstm_pred_do, _, _, _ = get_data(lake_id, 'ealstm', '2024-08-10-22-50', stage = 3, depth=n_depth)

        transformer_pred_do, _, _, _ = get_data(lake_id, 'transformer', '2024-08-10-02-35', stage = 3, depth=n_depth)


        dates_temp_pd = pd.to_datetime(dates_temp, format='%Y-%m-%d')
        dates_do_pd = pd.to_datetime(dates_do, format='%Y-%m-%d')
        # Find the indices where the first day of each row matches between dates_temp and dates_do
        matching_indices_temp = []
        matching_indices_do = []

        for i, date_temp in enumerate(dates_temp_pd[:, 0]):
            for j, date_do in enumerate(dates_do_pd[:, 0]):
                if date_temp == date_do:
                    matching_indices_temp.append(i)
                    matching_indices_do.append(j)

        # Now filter the arrays based on these matching indices
        print(matching_indices_temp)
        print(matching_indices_do)
        # Filter the temperature data arrays based on the matching indices
        lstm_pred_temp = lstm_pred_temp[matching_indices_temp, :]
        filtered_obs_temp = obs_temp[matching_indices_temp, :]
        filtered_sim_temp = sim_temp[matching_indices_temp, :]
        filtered_dates_temp = dates_temp[matching_indices_temp, :]
        pgfm_pred_temp = pgfm_pred_temp[matching_indices_temp, :]
        ealstm_pred_temp = ealstm_pred_temp[matching_indices_temp, :]
        transformer_pred_temp = transformer_pred_temp[matching_indices_temp, :]

        # Filter the oxygen data arrays based on the matching indices
        lstm_pred_do = lstm_pred_do[matching_indices_do, :]
        filtered_obs_do = obs_do[matching_indices_do, :]
        filtered_sim_do = sim_do[matching_indices_do, :]
        filtered_dates_do = dates_do[matching_indices_do, :]
        pgfm_pred_do = pgfm_pred_do[matching_indices_do, :]
        ealstm_pred_do = ealstm_pred_do[matching_indices_do, :]
        transformer_pred_do = transformer_pred_do[matching_indices_do, :]

        do_threshold = 11
        valid_indices = np.where(~np.isnan(filtered_obs_do) & ~np.isnan(filtered_obs_temp))
        valid_indices_obs = np.where((~np.isnan(filtered_obs_do) & ~np.isnan(filtered_obs_temp)) & (filtered_obs_do <= do_threshold))
        valid_indices_sim = np.where((~np.isnan(filtered_obs_do) & ~np.isnan(filtered_obs_temp)) & (filtered_sim_do <= do_threshold))
        # Filter the data based on these valid indices

        valid_do_obs = filtered_obs_do[valid_indices_obs]
        valid_temp_obs = filtered_obs_temp[valid_indices_obs]

        valid_do_pred = lstm_pred_do[valid_indices]
        valid_temp_pred = lstm_pred_temp[valid_indices]

        valid_do_sim = filtered_sim_do[valid_indices_sim] # LSTM
        valid_temp_sim = filtered_sim_temp[valid_indices_sim]

        pgfm_pred_temp = pgfm_pred_temp[valid_indices]
        pgfm_pred_do = pgfm_pred_do[valid_indices]

        ealstm_pred_temp = ealstm_pred_temp[valid_indices]
        ealstm_pred_do = ealstm_pred_do[valid_indices]


        transformer_pred_temp = transformer_pred_temp[valid_indices]
        transformer_pred_do = transformer_pred_do[valid_indices]
        # Combine the DO and temperature values for convex hull calculation
        points_obs = np.column_stack((valid_do_obs, valid_temp_obs))
        # Calculate the convex hull
        hull_obs = ConvexHull(points_obs)

        # Combine the DO and temperature values for convex hull calculation
        points_sim = np.column_stack((valid_do_sim, valid_temp_sim))
        # Calculate the convex hull
        hull_sim = ConvexHull(points_sim)

        points_pgfm = np.column_stack((pgfm_pred_do, pgfm_pred_temp))
        # Calculate the convex hull
        hull_pgfm = ConvexHull(points_pgfm)


        points_lstm = np.column_stack((valid_do_pred, valid_temp_pred))
        # Calculate the convex hull
        hull_lstm = ConvexHull(points_lstm)

        points_ealstm = np.column_stack((ealstm_pred_do, ealstm_pred_temp))
        # Calculate the convex hull
        hull_ealstm = ConvexHull(points_ealstm)

        points_transformer = np.column_stack((transformer_pred_do, transformer_pred_temp))
        # Calculate the convex hull
        hull_transformer = ConvexHull(points_transformer)


        # Plotting the data
        plt.figure(figsize=(10, 6))

        # # Plot the observed data and its convex hull
        plt.scatter(valid_do_obs, valid_temp_obs, alpha=0.5, label='Observed', color='blue')
        # plt.fill(points_obs[hull_obs.vertices, 0], points_obs[hull_obs.vertices, 1], 'blue', alpha=0.2, label='')
        # hull_obs = ConvexHull(points_obs)
        # smooth_obs = smooth_boundary(points_obs, hull_obs, smoothing_factor=0.001)
        # plt.plot(smooth_obs[0], smooth_obs[1], 'blue', alpha=0.5, label='Observed Boundary')
        alpha = 0.1 
        concave_hull, _ = alpha_shape(points_obs, alpha)
        x, y = concave_hull.exterior.xy
        plt.plot(x, y, color='blue', label='Observed Alpha Shape Boundary', alpha=0.5)

        # Plot the simulated data
        plt.scatter(valid_do_sim, valid_temp_sim, alpha=0.5, label='Simulated', color='red', marker='o')
        plt.fill(points_sim[hull_sim.vertices, 0], points_sim[hull_sim.vertices, 1], 'red', alpha=0.2, label='')

        # Plot the simulated data
        plt.scatter(valid_do_pred, valid_temp_pred, alpha=0.5, label='LSTM', color='purple', marker='o')
        plt.fill(points_lstm[hull_lstm.vertices, 0], points_lstm[hull_lstm.vertices, 1], 'purple', alpha=0.2, label='')


        plt.scatter(pgfm_pred_do, pgfm_pred_temp, alpha=0.5, label='FMPG', color='green', marker='o')
        plt.fill(points_pgfm[hull_pgfm.vertices, 0], points_pgfm[hull_pgfm.vertices, 1], 'green', alpha=0.2, label='')


        # plt.scatter(ealstm_pred_do, ealstm_pred_temp, alpha=0.5, label='ealstm', color='gray', marker='o')
        # plt.fill(points_ealstm[hull_ealstm.vertices, 0], points_ealstm[hull_ealstm.vertices, 1], 'gray', alpha=0.2, label='')

        # plt.scatter(transformer_pred_do, transformer_pred_temp, alpha=0.5, label='transformer', color='orange', marker='o')
        # plt.fill(points_transformer[hull_transformer.vertices, 0], points_transformer[hull_transformer.vertices, 1], 'orange', alpha=0.2, label='')

        save_path = os.path.join('../pics/relation_2', f'{lake_id}.pdf')
        plt.title(f'{lake_id}_index{index}_Scatter Plot of DO vs Temperature with Observed Envelope')
        plt.xlabel('Dissolved Oxygen (DO)')
        plt.ylabel('Temperature')
        plt.legend()
        plt.grid(True)
        # plt.show()
        plt.savefig(save_path)
    except Exception as e:
        print(f"Error occurred while processing lake_id {lake_id}: {e}")